In [1]:
import sys
sys.path.append('/home/jra21/work/source/repos/opep')

In [2]:
from openep import io as openep_io
from openep import case as openep_case
from openep import mesh_routines as openep_mesh
from openep import case_routines as case_routines
from openep import draw_routines as draw
import numpy as np

In [3]:
filename = '/home/jra21/work/source/repos/opep/examples/data/new_dataset_2.mat'
distance_thresh = 10

ep_case = openep_io.load_case(filename)

In [4]:
# Anatomic descriptions (Mesh) - nodes and indices
pts = ep_case.nodes
indices = ep_case.indices


In [5]:
# Electric data
# Locations – Cartesian co-ordinates, projected on to the surface 
locations = case_routines.get_electrogram_coordinates(ep_case,'type','bip')
i_egm = ep_case.electric['egm'].T
i_vp = case_routines.getMappingPointsWithinWoI(ep_case)
# macthing the shape of ivp with data
i_vp_egm = np.repeat(i_vp, repeats=i_egm.shape[1], axis=1)
# macthing the shape of ivp with coords
i_vp_locations = np.repeat(i_vp, repeats=locations.shape[1],axis=1)

# Replacing the values outside the window of interest with Nan values
i_egm[~i_vp_egm] = np.nan
locations[~i_vp_locations] = np.nan

# For each mapping point, n, find the voltage amplitude
max_volt = np.amax(a=i_egm,axis=1).reshape(len(i_egm),1)
min_volt = np.amin(a=i_egm,axis=1).reshape(len(i_egm),1)

amplitude_volt = np.subtract(max_volt,min_volt)

for indx in range(amplitude_volt.shape[1]):
    temp_data = amplitude_volt[:,indx]
    temp_coords = locations
    i_nan = np.isnan(temp_data)
    temp_data=temp_data[~i_nan]
    temp_coords=temp_coords[~i_nan]


    interp = case_routines.OpenEPDataInterpolator(method='rbf',distanceThreshold=distance_thresh,rbfConstant=1)
    vertex_voltage_data = interp.interpolate(x0=temp_coords,d0=temp_data,x1=pts)



vsurf = draw.DrawMap(ep_case,
        volt = vertex_voltage_data,
        freeboundary_color='black',
        cmap='jet_r',
        freeboundary_width=5,
        minval=0,
        maxval=2,
        volt_below_color='brown', 
        volt_above_color='magenta', 
        nan_color='gray', 
        plot=True,)

RBF Check
max|y - yi| = 5.659028801119348e-12
Perimeter is : 49.6852 | Area is: 154.7759
Perimeter is : 98.9257 | Area is: 634.7572
Perimeter is : 69.4603 | Area is: 331.5392
Perimeter is : 66.8485 | Area is: 260.1277
Perimeter is : 65.7998 | Area is: 282.0847
Perimeter is : 122.2539 | Area is: 949.7377
Perimeter is : 84.9425 | Area is: 508.7818


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [6]:
# import numpy as np

# def get_electrogram_coordinates(mesh_case, *args):
#     '''
    
#     '''
#     n_standard_args = 1
#     v_type = "bip"
    
#     nargin = len(args)+1
#     if nargin>n_standard_args:
#         for i in range(0,(nargin-n_standard_args),2):
#             if np.char.lower(args[i]) == "type":
#                 v_type = args[i+1]
    
#     if np.char.lower(v_type) == "bip":
#         x = mesh_case.electric["egmX"].T
        
#     elif np.char.lower(v_type) == "uni":
#         if not("egmUniX" in mesh_case.electric):
#             raise ValueError("There is no unipolar data associated with this openep case")
            
#         else:
#             x = mesh_case.electric['egmUniX'].T
        
#     return x

In [8]:
# Load EGM voltage values
# UseCase 28 - Get the electrogam at a mapping point #39

def get_egms_at_points(mesh_case,):
    pass